In [34]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime as dt

# Load S&P 500 companies list
spx = pd.read_excel('./data/sp500_list_14112024.xlsx')

def to_float(x):
    if type(x) == float or type(x) == int:
        return x
    if ',' in x:
        x = x.replace(',', '')
    if 'K' in x:
        if len(x) > 1:
            return float(x.replace('K', '')) * 1000
        return 1000.0
    if 'M' in x:
        if len(x) > 1:
            return float(x.replace('M', '')) * 1000000
        return 1000000.0
    if 'B' in x:
        return float(x.replace('B', '')) * 1000000000
    return 0.0

spx['Market Cap'] = spx['Market Cap'].apply(to_float)

# Download historical price data for selected ticker and indices
tickers = spx['Symbol'].unique().tolist()  # Get all S&P 500 tickers
indices = ['^GSPC', '^DJI', '^RUT']        # S&P 500, Dow Jones, Russell 2000 symbols
selected_stock = 'AAPL'  # Replace with the ticker you are analyzing

# Define timeframes
end_date = dt.datetime(2024, 9, 30)
data = yf.download(tickers + indices + [selected_stock], start=end_date - pd.DateOffset(months=13), end=end_date)['Adj Close']

# Extract stock and index data
stock_data = data[selected_stock]
index_data = data[indices]


[*********************100%***********************]  506 of 506 completed

2 Failed downloads:
['BRK.B']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
['BF.B']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2023-08-30 00:00:00 -> 2024-09-30 00:00:00)')


In [38]:
# Define function for financial metrics
def calculate_metrics(data, period_days, benchmark_data=None):
    rf = 0.045
    returns = data.pct_change().dropna()
    
    # Cumulative Returns
    cumulative_return = (data.iloc[-1] - data.iloc[-period_days]) / data.iloc[-period_days]
    
    # Sharpe Ratio
    sharpe_ratio = (returns[-period_days:].mean() - rf/252) / returns[-period_days:].std() * np.sqrt(252)
    
    # Volatility (Annualized)
    volatility = returns[-period_days:].std() * np.sqrt(252)
    
    # Maximum Drawdown
    rolling_max = data.rolling(window=period_days, min_periods=1).max()
    drawdown = (data / rolling_max) - 1
    max_drawdown = drawdown.min()
    
    # Correlation with Benchmark (if provided)
    if benchmark_data is not None:
        benchmark_returns = benchmark_data.pct_change().dropna()
        correlation = returns[-period_days:].corr(benchmark_returns[-period_days:])
    else:
        correlation = np.nan  # No correlation if no benchmark is provided
    
    return cumulative_return, sharpe_ratio, volatility, max_drawdown, correlation

# Initialize metrics DataFrames
metrics = pd.DataFrame()
periods = {'3M': 63, '6M': 126, '12M': 252}  # Trading days in 3, 6, and 12 months

# Calculate metrics for the individual stock and its correlation with indices
for period, days in periods.items():
    # Individual Stock
    stock_metrics = calculate_metrics(stock_data, days)  # No correlation here (stock_data alone)
    metrics[f'{selected_stock}_{period}'] = stock_metrics
    
    # S&P 500, Dow Jones, Russell 2000 Indices
    for index in indices:
        index_metrics = calculate_metrics(stock_data, days, benchmark_data=index_data[index])
        metrics[f'{index}_{period}'] = index_metrics

# Calculate metrics for peers (choose 3 from the same industry)
# Get the market cap of the selected stock
selected_market_cap = spx.loc[spx['Symbol'] == selected_stock, 'Market Cap'].values[0]

# Filter stocks in the same industry and exclude the selected stock
industry_peers = spx[(spx['GICS Sub-Industry'] == spx.loc[spx['Symbol'] == selected_stock, 'GICS Sub-Industry'].values[0]) &
                     (spx['Symbol'] != selected_stock)].copy()

# Calculate the absolute difference in market cap with the selected stock
industry_peers['Cap Difference'] = abs(industry_peers['Market Cap'] - selected_market_cap)

# Sort by market cap difference and select top 3 nearest peers
nearest_peers = industry_peers.sort_values(by='Cap Difference').head(3)['Symbol'].tolist()
peer_metrics = pd.DataFrame()

for peer in nearest_peers:
    peer_data = data[peer]
    for period, days in periods.items():
        peer_metrics[f'{peer}_{period}'] = calculate_metrics(peer_data, days, benchmark_data=stock_data)

# Display results
metrics.index = ['Cumulative Return', 'Sharpe Ratio', 'Volatility', 'Max Drawdown', 'Correlation with Benchmark']
peer_metrics.index = ['Cumulative Return', 'Sharpe Ratio', 'Volatility', 'Max Drawdown', 'Correlation with Selected Stock']

In [39]:
metrics

,AAPL_3M,^GSPC_3M,^DJI_3M,^RUT_3M,AAPL_6M,^GSPC_6M,^DJI_6M,^RUT_6M,AAPL_12M,^GSPC_12M,^DJI_12M,^RUT_12M
Cumulative Return,0.052151,0.052151,0.052151,0.052151,0.343075,0.343075,0.343075,0.343075,0.341354,0.341354,0.341354,0.341354
Sharpe Ratio,1.297190,1.297190,1.297190,1.297190,2.216477,2.216477,2.216477,2.216477,1.239504,1.239504,1.239504,1.239504
Volatility,0.231065,0.231065,0.231065,0.231065,0.252627,0.252627,0.252627,0.252627,0.221621,0.221621,0.221621,0.221621
Max Drawdown,-0.153548,-0.153548,-0.153548,-0.153548,-0.166067,-0.166067,-0.166067,-0.166067,-0.166067,-0.166067,-0.166067,-0.166067
Correlation with Benchmark,NaN,0.666941,0.520727,0.397836,NaN,0.563395,0.368065,0.364514,NaN,0.554755,0.385785,0.312180


In [40]:
peer_metrics

,DELL_3M,DELL_6M,DELL_12M,HPQ_3M,HPQ_6M,HPQ_12M,HPE_3M,HPE_6M,HPE_12M
Cumulative Return,-0.155322,0.012255,0.785897,0.021002,0.184260,0.432751,-0.041394,0.137490,0.175129
Sharpe Ratio,-0.871736,0.423496,1.226423,0.255073,1.017035,1.210239,-0.308049,0.812173,0.589174
Volatility,0.514468,0.587708,0.569651,0.285052,0.359466,0.291850,0.418794,0.377781,0.364187
Max Drawdown,-0.507869,-0.507869,-0.507869,-0.159801,-0.159801,-0.159801,-0.262820,-0.262820,-0.262820
Correlation with Selected Stock,0.442078,0.206618,0.148075,0.344670,0.209804,0.196876,0.351882,0.248054,0.142382


In [ ]:
# prompt for LLM, comparison, factual data etc, refer to prompt summary example